# Fluorescence per phase

This module allows a calculations for a second fluorescence channel, based on cells that have been binned into cell cycle phases. There is also an option to ignore the phase information.

In [1]:
import os
import re
import string
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from loguru import logger
from GEN_Utils import FileHandling

2019-08-04 16:07:07,778 GEN_Utils.FileHandling: [INFO ]  Import ok


### Set some sample-specific parameters

In [3]:
input_folder = 'examples/python/gauss_models/normalised/'
output_folder = 'examples/python/phase_fluorescence/'
fluorescence_col = 'TPE'
plate_samples = ['TPE only', '1', '1.5', '2', '3', '4']*4

plate_cords = [f'{x}{y}' for x in string.ascii_uppercase[0:4]
               for y in range(1, 7)]

sample_map = dict(zip(plate_cords, plate_samples))

In [4]:
if not os.path.exists(output_folder):
    os.mkdir(output_folder)
# Generate filelist

file_list = [filename for filename in os.listdir(input_folder)]

### Collect important info into summary df, grouped according to phase

In [6]:
sample_data = []

for filename in file_list:
    sample_name = os.path.splitext(filename)[0]
    raw_data = pd.read_csv(f'{input_folder}{filename}')
    raw_data.rename(columns={fluorescence_col: "fluorescence"}, inplace=True)
    fluo_data = raw_data.copy()[['phase', 'fluorescence']]
    fluo_data = fluo_data.groupby('phase').median().T
    fluo_data['sample'] = sample_name
    sample_data.append(fluo_data)

summary_df = pd.concat(sample_data).reset_index(drop=True)

In [7]:
summary_df.head()

phase,G,M,S,sample
0,6195.419922,9249.239258,7484.399902,1A2
1,4857.719727,7253.819824,5872.439941,1A3
2,4936.679688,7276.919922,5948.040039,1A4
3,3423.839844,4680.059814,3953.459839,1A5
4,3194.939941,4852.679688,3920.279785,1A6


In [8]:
summary_df['plate'] = summary_df['sample'].str[0]
summary_df['well'] = summary_df['sample'].str[1:]
summary_df['sample'] = summary_df['well'].map(sample_map)
summary_df.sort_values(['sample'], inplace=True)

In [9]:
summary_df.head(10)

phase,G,M,S,sample,plate,well
0,6195.419922,9249.239258,7484.399902,1,1,A2
10,5478.479980,8105.159668,6631.799805,1,1,C2
15,6417.599609,9706.619629,7822.919922,1,1,D2
5,5721.239746,8595.719727,6993.000000,1,1,B2
22,172.199997,1651.439941,10066.559570,1,2,B2
25,3061.799805,4431.000000,3668.279785,1.5,2,C3
6,4149.600098,6078.239746,5050.080078,1.5,1,B3
23,2551.079834,3730.439941,3041.219849,1.5,2,B3
1,4857.719727,7253.819824,5872.439941,1.5,1,A3
11,5121.479980,7557.479980,6219.359863,1.5,1,C3


### Generate equivalent dataset, ignoring phase

In [11]:
sample_data = {}

for filename in file_list:
    sample_name = os.path.splitext(filename)[0]
    raw_data = pd.read_csv(f'{input_folder}{filename}')
    raw_data.rename(columns={fluorescence_col: "fluorescence"}, inplace=True)
    fluo_data = raw_data.copy()['fluorescence']
    sample_data[sample_name] = fluo_data.median()

summary_df = pd.DataFrame.from_dict(sample_data, orient='index').reset_index()

summary_df.rename(columns={'index': 'sample',
                           0: 'med_fluorescence'}, inplace=True)

summary_df['plate'] = summary_df['sample'].str[0]
summary_df['well'] = summary_df['sample'].str[1:]
summary_df['sample'] = summary_df['well'].map(sample_map)
summary_df.sort_values(['plate', 'sample'], inplace=True)

In [12]:
summary_df.head(10)

,sample,med_fluorescence,plate,well
0,1,7271.039551,1,A2
5,1,6796.019775,1,B2
10,1,6478.919922,1,C2
15,1,7661.639648,1,D2
1,1.5,5672.520020,1,A3
6,1.5,4912.319824,1,B3
11,1.5,6021.959961,1,C3
16,1.5,6608.279785,1,D3
2,2,5820.359863,1,A4
7,2,3849.719971,1,B4
